### Memory Limitations

This notebook discusses the xxx

In [1]:
from pathlib import Path
root = Path("./data") # Set a data root path with enough disk space (>100MB)

### Download data if necessary

Downloads can take time, only execute this cell if you have not previously downloaded the data

In [2]:
from utils.download import download_single_vpu_data
download_single_vpu_data(root)

### Parameters

In [3]:
import pandas as pd
import xarray as xr

from diffhydro import (LTIRouter, LTIStagedRouter, TimeSeriesThDF,
                       CatchmentInterpolator, StagedCatchmentInterpolator)
from diffhydro.io import read_rapid_graph, read_multiple_rapid_graphs

In [4]:
# graph partitioning parameters
plength_thr = 10**4
node_thr = 10**4
# Routing model parameterss
max_delay = 32
dt = 1/24
# Experiment pathes and variables
device = "cuda:0"

In [5]:
rapid_config_path = root / 'geoglows' / 'rapid_config' / '305'
runoff_path = root / 'geoglows' / 'input' / '305_daily_sparse_runoff.pkl'
interp_weight_path = root / 'geoglows' / 'input' / '305_interp_weight.pkl'

### Route without sub-cluster stages

In [8]:
# Load the routing graph defined as a RAPID project into a RivTree structure
g = read_rapid_graph(rapid_config_path)

# Load input runoff.
# Here the data is provided as pixel-wise runoffs.
pixel_runoff = pd.read_pickle(runoff_path)  / (3600. * 24) # Convert values in m3 / s
pixel_runoff = TimeSeriesThDF.from_pandas(pixel_runoff).to(device) # Convert pandas DataFrame to TimeSeriesThDF

# Interpolate the pixel-wise runoffs onto the graph catchments 
interp_df = pd.read_pickle(interp_weight_path)
cat = CatchmentInterpolator(g, pixel_runoff, interp_df).to(device)
runoff = cat(pixel_runoff)

In [1]:
#runoff = cat(pixel_runoff)

In [ ]:
model = LTIRouter(max_delay=max_delay, dt=dt).to(device)

In [ ]:
discharges = model(runoff, g)

In [ ]:
discharges = discharges.to_pandas()
discharges[].plot()

### Route with sub-cluster stages